In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class HeatEquationMacCormack:
    """
    HeatEquationMacCormack
    ----------------------
    A solver for the 1-D heat equation using a predictor–corrector
    (MacCormack-style) discretization scheme.

    Governing PDE:
        ∂h/∂t = k ∂²h/∂x²

    Domain:
        0 <= x <= L

    Boundary conditions:
        h(0, t) = 0
        h(L, t) = sin(pi * t / 10)

    Initial condition:
        h(x, 0) = 0

    Parameters
    ----------
    L : float
        Length of the spatial domain (default 100).
    dx : float
        Spatial step size (default 1).
    dt : float
        Time step size (default 0.01).
    k : float
        Diffusion coefficient (default 1.0).

    Attributes
    ----------
    x : ndarray
        Discrete spatial grid points.
    h : ndarray
        Current solution vector at all spatial points.
    Nx : int
        Number of spatial grid points.

    Methods
    -------
    boundary_conditions(t):
        Apply boundary conditions at time t.
    step(t):
        Perform one MacCormack predictor–corrector step.
    run(T):
        Run simulation up to time T and return solution.
    plot_time_snapshots(times):
        Plot solution at specified times.
    plot_varying_k(ks, T):
        Plot solution at fixed time T for different k values.
    """

    def __init__(self, L=100, dx=1, dt=0.01, k=1.0):
        # Store parameters
        self.L = L
        self.dx = dx
        self.dt = dt
        self.k = k

        # Create spatial grid
        self.x = np.arange(0, L+dx, dx)
        self.Nx = len(self.x)

        # Initialize solution with zeros (initial condition)
        self.h = np.zeros(self.Nx)

    def boundary_conditions(self, t):
        """
        Apply boundary conditions at time t:
        - Left boundary fixed at 0
        - Right boundary oscillates as sin(pi * t / 10)
        """
        self.h[0] = 0
        self.h[-1] = np.sin(np.pi * t / 10)

    def step(self, t):
        """
        Perform one MacCormack step:
        Predictor: forward Euler using current h
        Corrector: average of predictor and corrected update
        """
        h = self.h.copy()

        # Predictor step
        h_pred = h.copy()
        for i in range(1, self.Nx-1):
            h_pred[i] = h[i] + self.dt * self.k * (h[i+1] - 2*h[i] + h[i-1]) / (self.dx**2)

        # Apply boundary conditions to predictor
        h_pred[0] = 0
        h_pred[-1] = np.sin(np.pi * t / 10)

        # Corrector step
        h_corr = h.copy()
        for i in range(1, self.Nx-1):
            h_corr[i] = 0.5 * (h[i] + h_pred[i] +
                               self.dt * self.k * (h_pred[i+1] - 2*h_pred[i] + h_pred[i-1]) / (self.dx**2))

        # Apply boundary conditions to corrector
        h_corr[0] = 0
        h_corr[-1] = np.sin(np.pi * t / 10)

        # Update solution
        self.h = h_corr

    def run(self, T):
        """
        Run simulation up to time T.
        Returns the solution h(x, T).
        """
        steps = int(T / self.dt)
        t = 0
        for _ in range(steps):
            self.step(t)
            t += self.dt
        return self.h

    def plot_time_snapshots(self, times):
        """
        Plot solution at specified times.
        Each run starts from the initial condition.
        """
        plt.figure(figsize=(8,6))
        for T in times:
            self.h[:] = 0  # reset to initial condition
            sol = self.run(T)
            plt.plot(self.x, sol, label=f"t={T}")
        plt.xlabel("x")
        plt.ylabel("h(x,t)")
        plt.title("Heat Equation with MacCormack Scheme")
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_varying_k(self, ks, T=20):
        """
        Plot solution at fixed time T for different diffusion coefficients k.
        """
        plt.figure(figsize=(8,6))
        for k in ks:
            self.k = k
            self.h[:] = 0  # reset to initial condition
            sol = self.run(T)
            plt.plot(self.x, sol, label=f"k={k}")
        plt.xlabel("x")
        plt.ylabel("h(x,T)")
        plt.title(f"Heat Equation at t={T} for varying k")
        plt.legend()
        plt.grid(True)
        plt.show()


# Example usage:
solver = HeatEquationMacCormack()
solver.plot_time_snapshots([0,4,8,12,16,20,24,30,50])
solver.plot_varying_k([0.1, 0.5, 1.0, 2.0], T=20)